In [1]:
import sqlite3
import datetime, requests, io, os, time, random, re
import pandas as pd
from BeautifulSoup import BeautifulSoup
from threading import Thread
import demjson
from lxml import html
from tqdm import tqdm_notebook

In [2]:
import constants

ImportError: No module named constants

In [4]:
conn = sqlite3.connect('./data/valuefinder.db')

In [ ]:
conn.execute('''CREATE TABLE history
         (ID INTEGER PRIMARY KEY  autoincrement,
         symbol           TEXT    NOT NULL,
         price            REAL     NOT NULL,
         traded_quanity            INTEGER     NOT NULL,
         no_of_trade            INTEGER     NOT NULL,
         delivery_percentage            REAL     NOT NULL,
         date         datetime, CONSTRAINT unq UNIQUE (symbol, date));''')
# conn.execute("DROP table history")

In [ ]:
conn.execute('''CREATE TABLE symbol
         (ID INTEGER PRIMARY KEY  autoincrement,
         symbol           TEXT    NOT NULL,
         count            INTEGER     NOT NULL,
         CONSTRAINT unq UNIQUE (symbol));''')


In [ ]:
conn.execute('''CREATE TABLE results
(
    symbol  INTEGER    NOT NULL,
    format TEXT NOT NULL,
    from_date datetime NOT NULL,
    to_date datetime NOT NULL,
    published_date datetime NOT NULL,
    sequence_number INTEGER NOT NULL,
    isAudited  INTEGER NOT NULL,
    isCumulative  INTEGER NOT NULL,
    isConsiladated  INTEGER NOT NULL,
    period TEXT NOT NULL,
    url TEXT NOT NULL,
    

    total_tax REAL NOT NULL DEFAULT 0,
    pl_attributable_to_non_controlling_interests REAL NOT NULL DEFAULT 0,
    pl_from_continuing_operations REAL NOT NULL DEFAULT 0,
    debenture_redemption_reserve REAL NOT NULL DEFAULT 0,
    consolidated_basic_eps REAL NOT NULL DEFAULT 0,
    face_value_of_debt REAL NOT NULL DEFAULT 0,
    face_value REAL NOT NULL DEFAULT 0,
    other_expense REAL NOT NULL DEFAULT 0,
    reserve_excluding_revaluation REAL NOT NULL DEFAULT 0,
    share_of_pl REAL NOT NULL DEFAULT 0,
    ci_attributable_to_non_controlling_interests REAL NOT NULL DEFAULT 0,
    finance_cost REAL NOT NULL DEFAULT 0,
    purchase_of_stock_in_trade REAL NOT NULL DEFAULT 0,
    paid_up_equity_shatre_capital REAL NOT NULL DEFAULT 0,
    pbt REAL NOT NULL DEFAULT 0,
    depreciation_expense REAL NOT NULL DEFAULT 0,
    pl_attributable_to_owners_of_parent REAL NOT NULL DEFAULT 0,
    pat REAL NOT NULL DEFAULT 0,
    ci_attributable_to_owners_of_parent REAL NOT NULL DEFAULT 0,
    changes_in_inventories REAL NOT NULL DEFAULT 0,
    consolidated_diluted_eps REAL NOT NULL DEFAULT 0,
    tax_expense_of_discontinued_operations REAL NOT NULL DEFAULT 0,
    employee_benifit_expense REAL NOT NULL DEFAULT 0,
    revenue_from_operations REAL NOT NULL DEFAULT 0,
    total_income REAL NOT NULL DEFAULT 0,
    cost_of_material_consumed REAL NOT NULL DEFAULT 0,
    pl_from_discontinued_operations REAL NOT NULL DEFAULT 0,
    other_income REAL NOT NULL DEFAULT 0,
    total_expense REAL NOT NULL DEFAULT 0,
    paid_up_debt_capital REAL NOT NULL DEFAULT 0,
    total_comprehensive_income REAL NOT NULL DEFAULT 0,
    current_tax REAL NOT NULL DEFAULT 0,
    deferred_tax REAL NOT NULL DEFAULT 0,
    
    
    india_holding_percentage REAL NOT NULL DEFAULT 0,
    npa_percentage REAL NOT NULL DEFAULT 0,
    provisions_and_contingencies REAL NOT NULL DEFAULT 0,
    interest_earned REAL NOT NULL DEFAULT 0,
    discount_on_advances REAL NOT NULL DEFAULT 0,
    income_on_investments REAL NOT NULL DEFAULT 0,
    capital_adequacy_ratio REAL NOT NULL DEFAULT 0
            
);
''')

## Fetch equities symbols and Insert


In [ ]:
all_equities_url = "https://www.nseindia.com/products/content/sec_bhavdata_full.csv"
all_equities = pd.read_csv(io.StringIO(requests.get(all_equities_url).content.decode('utf-8')))
all_equities = all_equities.loc[all_equities[' SERIES'] == " EQ"]
all_equities[" SERIES"] = "EQ"
all_equities = all_equities.filter(["SYMBOL"], axis=1)
len(all_equities)

def fetch_count(symbol):
    count_url = 'https://www.nseindia.com/marketinfo/sym_map/symbolCount.jsp?symbol=' + symbol
    headers = {'Referer': 'https://www.nseindia.com/products/content/equities/equities/eq_security.htm'}
#     print count_url
    request_content = requests.get(
        count_url,
        headers=headers
    ).content
    request_content = re.sub(r'\n', '', request_content)
    request_content = re.sub(r'\s', '', request_content)
    return request_content

all_equities['count'] = all_equities['SYMBOL'].map(fetch_count)


In [ ]:
all_equities.to_csv("../data/equities.csv", index=False, header=False)


In [ ]:
symbol_file_path = "../data/equities.csv"
symbol_records = pd.read_csv(symbol_file_path)
for symbol_record in symbol_records.iterrows():
    query = "insert into  symbol (symbol, count) values('{0}', '{1}');".format(symbol_record[1]["SYMBOL"], symbol_record[1]["COUNT"])
    conn.execute(query)
conn.commit()

## Fetch Price history

In [4]:
def fetch_equity_history(symbol, start_date, end_date, symbol_count, symbol_id):
    history_equity_url = "https://www.nseindia.com/products/dynaContent/common/productsSymbolMapping.jsp?symbol={0}&segmentLink=3&symbolCount={3}&series=EQ&dateRange=+&fromDate={1}&toDate={2}&dataType=PRICEVOLUMEDELIVERABLE"
#     print "Fetching for {0} {1} {2}".format(symbol, start_date, end_date)
    headers = {'Referer': 'https://www.nseindia.com/products/content/equities/equities/eq_security.htm'}
    parsed_html = BeautifulSoup(
        requests.get(
            history_equity_url.format(
                symbol, 
                start_date, 
                end_date,
                symbol_count
            ),
            headers=headers
        ).content
    )
#     print history_equity_url.format(
#                 symbol, 
#                 start_date, 
#                 end_date,
#                 symbol_count
#     );
#     print parsed_html.find("div", {"id": "csvContentDiv"}).text
    try:
        year_data = pd.read_csv(
            io.StringIO(
                parsed_html.find("div", {"id": "csvContentDiv"}).text.replace(":", "\n").decode('utf-8')
            )
        )
#         print "Fetch success for {0} {1} {2} {3}".format(symbol, start_date, end_date, symbol_id)
    except:
#         print "---------created dummy df for {0}-------".format(symbol)
        return pd.DataFrame()

#     print "pd obj created  for {0} {1} {2}".format(symbol, start_date, end_date)
    year_data = year_data.drop(year_data.columns[[0, 1, 3, 7, 9, 11, 13]], axis=1)
    insert_equity_history(year_data, symbol_id)
    
    return year_data
    



In [5]:
def insert_equity_history(data, symbol_id):
    conn = sqlite3.connect('../data/NSE/equities/valuefinder.db')    
    for row in data.iterrows():
        date =  datetime.datetime.strptime(row[1]["Date"], "%d-%b-%Y").strftime("%Y-%m-%d")
        close_price =  row[1]["Close Price"]
        traded_quanity=  row[1]["Total Traded Quantity"]
        no_of_trade=  row[1]["No. of Trades"]
        delivery_percentage=  row[1]["% Dly Qt to Traded Qty"]
        query = '''insert into  history 
            (symbol, price, date, delivery_percentage, no_of_trade, traded_quanity) 
            values('{0}', '{1}', '{2}', '{3}', '{4}', '{5}');'''.format(
                symbol_id, 
                close_price, 
                date, 
                delivery_percentage, 
                no_of_trade, 
                traded_quanity
        )
#         print query
        try:
            conn.execute(query)
        except:
            continue
    conn.commit()
    
            
            

In [ ]:

def get_recent_history():
    symbols = conn.execute("SELECT * from symbol").fetchall()
    threads = []
    
    for symbol in symbols:
        symbol_id = symbol[0]
        symbol_name = symbol[1]
        count = symbol[2]
        last_fetched_date = conn.execute("SELECT max(date) FROM history where symbol={}".format(symbol_id)).fetchall()[0][0]
        last_fetched_date = datetime.datetime.strptime(last_fetched_date, '%Y-%m-%d').strftime('%d-%m-%Y') if last_fetched_date is not None else "01-01-2019"
        end_date = datetime.datetime.now().strftime('%d-%m-%Y')
        if last_fetched_date < end_date:
            thread = Thread(
                target = fetch_equity_history, 
                args = (symbol_name, last_fetched_date, end_date, count,  symbol_id)
            )
            threads.append(thread)
            
    
    
    for x in threads:
        x.start()

    for x in tqdm_notebook(threads):
        time.sleep(0.1)
        x.join()

get_recent_history()    


In [6]:
def get_year_history(year):
    symbols = conn.execute("SELECT * from symbol").fetchall()
    threads = []

    for symbol in symbols:
        symbol_id = symbol[0]
        symbol_name = symbol[1]
        count = symbol[2]
        
        for month in [1, 2]:
            start_date = "01-%s-%d" % ("01" if month is 1 else "08", year)
            end_date = "31-%s-%d" % ("07" if month is 1 else "12", year)
            thread = Thread(
                target = fetch_equity_history, 
                args = (symbol_name, start_date, end_date, count,  symbol_id)
            )
            threads.append(thread)
            

    for x in tqdm_notebook(threads):
        x.start()
        time.sleep(0.2)
        x.join()
    
get_year_history(2011)

# print "-----------> Fetch 2012"
# get_year_history(2012)

## Fetch Quarterly results


In [14]:
def identify_format(html_response):
    for format_id in constants.format_identifier_mapping:
        should_have_strings = constants.format_identifier_mapping[format_id]["should_have"]
        available_strings = filter(lambda x: html_response.find(x) > -1, should_have_strings)
        if(len(available_strings) == len(should_have_strings)):
            if len(constants.format_identifier_mapping[format_id].get("should_not_have", [])):
                should_not_have_strings = constants.format_identifier_mapping[format_id]["should_not_have"]
                not_available_strings = filter(lambda x: html_response.find(x) < 0, should_not_have_strings)
                if(len(not_available_strings) == len(should_not_have_strings)):
                    return format_id
            else: 
                return format_id

        
last_24_months_url = "https://www.nseindia.com/corporates/corpInfo/equities/getFinancialResults.jsp?broadcastPeriod=Last%2024%20Months&symbol={}&industry=&period=Quarterly"
more_than_24_months_url = "https://www.nseindia.com/corporates/corpInfo/equities/getFinancialResults.jsp?start=0&limit=200&broadcastPeriod=More%20than%2024%20Months&symbol={}&industry=&period=Quarterly"
annual_last_24_months_url = "https://www.nseindia.com/corporates/corpInfo/equities/getFinancialResults.jsp?broadcastPeriod=Last%2024%20Months&symbol={}&industry=&period=Annual"
annual_more_than_24_months_url =  "https://www.nseindia.com/corporates/corpInfo/equities/getFinancialResults.jsp?broadcastPeriod=More%20than%2024%20Months&symbol={}&industry=&period=Annual"


def get_results_metadata(symbol):
    last_24_months_response = requests.get(last_24_months_url.format(symbol))
    more_than_24_months_response = requests.get(more_than_24_months_url.format(symbol))
    
    annual_last_24_months_response = requests.get(annual_last_24_months_url.format(symbol))
    annual_more_than_24_months_response = requests.get(annual_more_than_24_months_url.format(symbol))
    return demjson.decode(last_24_months_response.content)["rows"] + \
            demjson.decode(more_than_24_months_response.content)["rows"] + \
            demjson.decode(annual_last_24_months_response.content)["rows"] + \
            demjson.decode(annual_more_than_24_months_response.content)["rows"]



def get_quater(result_metadata):
    month = result_metadata["FromDate"].split(" ")[0].split("-")[1]
    if (month == "Apr"):
        return "Q1"
    elif (month == "Jul"):
        return "Q2"
    elif (month == "Oct"):
        return "Q3"
    elif (month == "Jan"):
        return "Q4"
    
def generate_result_url(result_metadata):
    url_params = {
        "param": result_metadata["FromDate"] + 
                 result_metadata["ToDate"] +
                ("AN" if result_metadata["Period"] == "Annual" else get_quater(result_metadata))+
                ("A"  if result_metadata["Audited"] == "Audited" else "U")+
                result_metadata["Bank"] +
                ("C"  if result_metadata["Cumulative"] == "Cumulative" else "N")+
                ("C" if result_metadata["Consolidated"] == "Consolidated" else "N") +
                ("A" if result_metadata["IND_AS"] == "Non-Ind-AS" else "N") +
                "E" +
                result_metadata["Symbol"],
        "seq_id": result_metadata["SeqNumber"],
        "industry": result_metadata["Ind"],
        "frOldNewFlag": result_metadata["fr_oldNewFlag"],
        "viewFlag": "N"
        
    }
    if result_metadata["Bank"] == "N":
        if convert_to_float(result_metadata["ToDate"].split("-")[2]) > 2016:
            url_pattern = "https://www.nseindia.com/corporates/corpInfo/equities/results_Nxbrl.jsp?param={param}&seq_id={seq_id}&industry={industry}&viewFlag={viewFlag}&frOldNewFlag=null"
        else:
            url_pattern = "https://www.nseindia.com/corporates/corpInfo/equities/results.jsp?param={param}&seq_id={seq_id}&industry={industry}&viewFlag={viewFlag}&frOldNewFlag=null"
    else:
        url_pattern = "https://www.nseindia.com/corporates/corpInfo/equities/results.jsp?param={param}&seq_id={seq_id}&industry={industry}&viewFlag={viewFlag}&frOldNewFlag=null"
        
    return url_pattern.format(**url_params)

def fetch_results(result_url):
    html_response = requests.get(result_url)
    html_response = html_response.content.replace("\r", "").replace("\n", "").replace("<br>", "")
    html_response = html_response.replace("<p>", "").replace("</p>", "")
    return html_response

def convert_to_float(val):
    try:
        return float(val)
    except:
        return 0.0

def parse_result(format_id, html_response, result_metadata, symbol_id, url):
    result = {}
    
    result["symbol"] = symbol_id
    
    result["period"] = get_quater(result_metadata) if result_metadata["Period"] != "Annual" else "AN"
    result["format"] = format_id
    result["from_date"] = datetime.datetime.strptime(result_metadata["FromDate"], "%d-%b-%Y").strftime("%Y-%m-%d")
    result["to_date"] = datetime.datetime.strptime(result_metadata["ToDate"], "%d-%b-%Y").strftime("%Y-%m-%d")
    result["published_date"] = datetime.datetime.strptime(result_metadata["FilingDate"].split(" ")[0], "%d-%b-%Y").strftime("%Y-%m-%d")
    
    result["sequence_number"] = result_metadata["SeqNumber"]
    result["isAudited"] = 1 if result_metadata["Audited"] == "Audited" else 0
    result["isCumulative"] = 1 if result_metadata["Cumulative"] == "Cumulative" else 0
    result["isConsiladated"] = 1 if result_metadata["Consolidated"] == "Consolidated" else 0
    result["url"] = url
    
    tree = html.fromstring(html_response)
    for i in range(1, 70):
        text = tree.cssselect(constants.field_mappinng[format_id]["XPath"].format(i))
        if len(text):
            text = text[0].text_content()
            text = " ".join(text.split("  "))
            for key in constants.field_mappinng[format_id]["fields"]:
                if constants.field_mappinng[format_id]["fields"][key] in text:
                    text = "".join(text.split(constants.field_mappinng[format_id]["fields"][key]))
                    value = 0 if len(re.findall(r'[+-]?\d+\.\d+|[+-]?\d+', text)) == 0 else re.findall(r'[+-]?\d+\.\d+|[+-]?\d+', text)[0]
                    result[key] = value if result.get(key, False) == False else result[key]
                    break
    return result



def compose_fetch_result_fns(symbol):
    meta_data = get_results_metadata(symbol)
    symbol_id = conn.execute("SELECT * from symbol where symbol='{0}'".format(symbol)).fetchall()[0][0]
    available_sequnce_numbers = conn.execute("SELECT sequence_number from results where symbol='{0}'".format(symbol_id)).fetchall()
    available_sequnce_numbers = map(lambda x: x[0], available_sequnce_numbers)
    meta_data = filter(lambda x: int(x["SeqNumber"]) not in available_sequnce_numbers, meta_data)

    threads = []
    for meta_datum in meta_data:
        thread = Thread(
            target = fetch_from_meta, 
            args = (meta_datum, symbol_id)
        )
        threads.append(thread)
    
    for x in threads:
        x.start()

    for x in threads:
        x.join()
    
    
    
def fetch_from_meta(meta_datum, symbol_id):
    url = generate_result_url(meta_datum)
    html_response = fetch_results(url)
    format_id = identify_format(html_response)
    if format_id is not None:
        parsed_result = parse_result(format_id, html_response, meta_datum, symbol_id, url)
        insert_result(parsed_result, format_id)
    
def generate_insert_query(format_id):
    
    meta_fields = "symbol, format, from_date, to_date, published_date, sequence_number, isAudited, isCumulative, isConsiladated, period, url"
    meta_values = "'{symbol}', '{format}', '{from_date}', '{to_date}', '{published_date}', '{sequence_number}', '{isAudited}', '{isCumulative}', '{isConsiladated}', '{period}', '{url}'"

    query = "INSERT INTO results ("
    for key in constants.field_mappinng[format_id]["fields"]:
        if key != "IGNORE_THIS":
            query = query + key + ", "
        
    query = query + meta_fields + ") VALUES ("
    
    for key in constants.field_mappinng[format_id]["fields"]:
        if key != "IGNORE_THIS":
            query = query +"'{" + key + "}', "
    
    query = query + meta_values + ")"
    return query

    
    
def insert_result(result, format_id):
    conn = sqlite3.connect('../data/NSE/equities/valuefinder.db')    
    query_template = generate_insert_query(format_id)
    query = query_template.format(**result)
    try:
        conn.execute(query)
    except:
        pass
    conn.commit()

    
    
    

    

In [15]:

def get_all_company_results():
    symbols = conn.execute("SELECT * from symbol").fetchall()
    for symbol in tqdm_notebook(symbols):
        time.sleep(0.1)
        compose_fetch_result_fns(symbol[1])
        

get_all_company_results()

KeyboardInterrupt: 